# Movie Recommendations

## From Scratch

https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/

In [1]:
import pandas as pd
import numpy as np

In [3]:
# reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('datasets/ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('datasets/ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

# reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('datasets/ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

In [4]:
print("\nUser Data :")
print("shape : ", users.shape)
print(users.head())


User Data :
shape :  (943, 5)
   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


We have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code.

In [5]:
# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
print(ratings.head())


Ratings Data :
shape :  (100000, 4)
   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596


We have 100k ratings for different user and movie combinations. Now finally examine the items file.

In [7]:
# Item Data
print("\nItem Data :")
print("shape : ", items.shape)
display(items.head())


Item Data :
shape :  (1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


This dataset contains attributes of 1682 movies. There are 24 columns out of which last 19 columns specify the genre of a particular movie. 

The dataset has already been divided into train and test by GroupLens where the test data has 10 ratings for each user, i.e. 9,430 rows in total. We will read both these files into our Python environment.

In [9]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('datasets/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('datasets/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [10]:
# Get number of users and number of items
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [11]:
# Create user-item matrix, which can be used to calculate simularity
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [12]:
# Calculate similarity
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

This gives us the item-item and user-user similarity in an array form.

In [13]:
# Define function to make predictions based on similarities
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

Make predictions based on user similarity and item similarity.

In [14]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

## Using Surprise

### Load data

In [33]:
df = pd.read_csv('datasets/ml-100k/csvratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [34]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [36]:
# number of unique users
len(set(df['userId']))

610

In [37]:
# number of unique movies
len(set(df['movieId']))

9724

In [38]:
# Drop unnecessary columns
new_df = df.drop(columns='timestamp')

In [39]:
# Transform data to be compatible with surprise
# Need reader and dataset class
# specific load_from_df method to load dataframes

from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_df,reader)

Investigate number of unsers and items in dataset.

In [40]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


### Modelling

In [41]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

Try Singular Value Decomposition with gridsearch.

In [42]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}

g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [43]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8699146741014931, 'mae': 0.6686410046868377}
{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


Try KNNBasic with cross validation.

In [44]:
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.96411362, 0.9818695 , 0.98152191, 0.97149668, 0.96717549]))
('test_mae', array([0.7430703 , 0.75739582, 0.75659792, 0.75083855, 0.74738615]))
('fit_time', (0.3607501983642578, 0.5151412487030029, 0.47708916664123535, 0.35434699058532715, 0.3688020706176758))
('test_time', (1.3034820556640625, 1.3033621311187744, 1.17868971824646, 1.3411879539489746, 1.2850849628448486))
-----------------------
0.9732354407301195


In [45]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [46]:
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87270223, 0.87092672, 0.88055907, 0.87991094, 0.87353519]))
('test_mae', array([0.66739345, 0.66261186, 0.67129278, 0.67404457, 0.66643656]))
('fit_time', (0.6814899444580078, 0.7372121810913086, 0.8047440052032471, 0.7301099300384521, 0.6402249336242676))
('test_time', (2.0350148677825928, 2.354997158050537, 2.003427028656006, 2.072472095489502, 2.205176830291748))


0.8755268301533794

### Generating predictions

Need to load dataset which has movie titles.

In [50]:
df_movies = pd.read_csv('datasets/ml-100k/csvmovies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [51]:
# Fit top model
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [52]:
# Generate prediction for user id 2, movie id 4
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.023120882527799, details={'was_impossible': False})

### I/O

In [53]:
# Define function to rate movies
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list 

In [54]:
# Sample output
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
               {'userId': 1000, 'movieId': 2491, 'rating': '4'},
               {'userId': 1000, 'movieId': 4718, 'rating': '4'},
               {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

Make predictions for this new user.

In [55]:
# Add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [56]:
# Train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [57]:
# Make predictions for the user
# Create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [58]:
# Order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [61]:
# Define function to return the top n recommendations
def recommended_movies(ranked_movies,movie_title_df,n):
        for idx, rec in enumerate(ranked_movies):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  901    Brazil (1985)
Name: title, dtype: object 

Recommendation #  3 :  909    Apocalypse Now (1979)
Name: title, dtype: object 

Recommendation #  4 :  2226    Fight Club (1999)
Name: title, dtype: object 

Recommendation #  5 :  659    Godfather, The (1972)
Name: title, dtype: object 

